In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import os

import numpy as np
import pandas as pd

import librosa
import IPython.display as ipd

from afritts_text_normalizer import tts_cleaner

In [7]:
train = pd.read_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/intron-tts-train-public-28565.csv")
dev = pd.read_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/intron-tts-dev-public-3330.csv")
test = pd.read_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/intron-tts-test-public-4161.csv")

data = pd.concat([train, dev, test])

# remove 3 missing files
data = data[data.audio_ids != "24bfceabfa102cc1c1926d2049f56bbf"]
data = data[data.audio_ids != "790cdbb7907d0112bc737f0b06282dde"]
data = data[data.audio_ids != "4bffcdb03a445616d70eb1a859ac52a5"]

# Remove utterances with long duration, mostly noisy
data = data[data.duration <= 50.0]

In [8]:
train.user_ids.nunique(), dev.user_ids.nunique(), test.user_ids.nunique()

(600, 76, 75)

In [9]:
data.user_ids.nunique(), data.duration.sum()/3600  # 751 speaker, 135 hours

(751, 133.57965694444445)

In [10]:
# data[data.duration > 50.0].duration.sum() / 3600

In [11]:
len(data), data.audio_ids.nunique()

(35995, 35995)

In [12]:
# percentage of utterances with numbers

import re
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

data["is_alnum"] = data.transcript.apply(lambda x: has_numbers(x))

In [13]:
data[data.is_alnum == True].duration.sum() / 3600 # 42 hours contain numbers, ~ 40%

41.340895

In [14]:
split = data # train, dev, test
speakers = pd.pivot_table(data, values="duration",
              columns=["user_ids"], aggfunc="sum").T.reset_index()

speakers["duration"] = speakers["duration"]# /3600

speakers = speakers.sort_values("duration", ascending=False).reset_index(drop=True)
speakers.head(20)

,user_ids,duration
0,2db35217e91d8889c6f6f4bdee66a52f,5002.340
1,73f971bbef38880a86ac97680ab5a7f8,4929.179
2,6b5cd67595796bf5489893b69755f43f,4834.860
3,1269ac57cb62784a2abeec0b904ed615,4796.768
4,b883f5d99b7dfc05fc7bbeca0f926e9f,4675.044
5,f5a8238265acd295b474e3b67cde1dd5,4665.078
6,96ca0f339408b47bbed4b4ed6fdc4460,4570.077
7,a35a75f5541bce9a15063660e1028215,4506.899
8,2a477d9f7d285736be740fe77180b9c0,4197.769
9,bef1fe43fb65dfc614c001baeef2a381,4123.003


In [15]:
speakers.tail(10)

,user_ids,duration
741,b545a4ca235a7b72688a1c0b3eb6bde6,8.181
742,2493cba45cdf00e461b67e78d518a910,7.706
743,633ea93e9a3eb22f92e6c522c11f1f52,7.559
744,df4a2d9487db6e98bf0d8825220f9bf4,7.391
745,ce339a60ecffc5a72db4bd2489d206cc,7.379
746,1a1b8d8aace7c96a29d6b32d74ed90fd,7.311
747,2bb382f192f6a57300bdb0ef6fbd8ad3,7.077
748,eb5df6c57ad707440696d8478e655d78,6.603
749,c73de2fe9d445a8d192c5aac49a0e0bb,5.773
750,6b76a3d99dd1fe093782ef197cb92816,5.035


In [24]:
speakers_dict = {}
for i, item in speakers.iterrows():
    speakers_dict[item.user_ids] = item.duration

In [16]:
accents = pd.pivot_table(data, values="duration",
              columns=["accent"], aggfunc="sum").T.reset_index()
accents["duration"] = accents["duration"]
accents = accents.sort_values("duration", ascending=False).reset_index(drop=True)

In [17]:
accents.head(10) # don't forget to drop unknown accent

,accent,duration
0,Yoruba,113126.218
1,Hausa,110536.472
2,Unknown,36896.805
3,Swahili,36781.011
4,Igbo,28398.575
5,Zulu,23158.020
6,Ijaw,17126.945
7,Idoma,10673.228
8,Igala,9229.985
9,Afrikaans,7482.533


In [18]:
accents.tail(20)

,accent,duration
60,Bajju,182.389
61,Tula,167.667
62,Dera,166.638
63,Chichewa,149.858
64,Tsonga,124.148
65,Ogbia,78.476
66,Swati,77.642
67,Otjiherero,61.700
68,Ikulu,54.912
69,Lunguda,42.362


In [19]:
split = data # train, dev, test
speakers2 = pd.pivot_table(data, values="duration",
              columns=["country", "accent", "gender"], aggfunc="sum").T.reset_index()

speakers2["duration"] = speakers2["duration"]# /3600

speakers2 = speakers2.sort_values("duration", ascending=False).reset_index(drop=True)
speakers2.head(20)

,country,accent,gender,duration
0,NG,Yoruba,Male,63328.080
1,NG,Hausa,Male,57076.143
2,NG,Hausa,Female,53003.214
3,NG,Yoruba,Female,49789.957
4,KE,Swahili,Female,23695.523
5,ZA,Zulu,Female,22767.538
6,NG,Igbo,Female,17832.832
7,NG,Ijaw,Male,11829.030
8,KE,Unknown,Female,10855.926
9,NG,Igbo,Male,10565.743


In [20]:
speakers2.tail(20)

,country,accent,gender,duration
101,NG,Ikulu,Female,54.912
102,NG,Pidgin,Female,53.226
103,KE,Kikuyu,Female,51.627
104,UG,Luganda,Female,42.840
105,NG,Lunguda,Male,42.362
106,NG,Etsako,Female,34.521
107,NG,Ogba,Female,34.218
108,NG,"Hausa, Fulani",Male,33.247
109,NG,Anaang,Male,31.140
110,ZA,Portuguese,Male,30.236


In [21]:
# Test set: 4 speakers with 4 different accents
test_data_unseen = data[data.accent.isin(["Swati", "Portuguese", "Ogbia", "Lunguda"])]

In [38]:
len(test_data_unseen)

16

In [23]:
data = data[~data.accent.isin(["Swati", "Portuguese", "Ogbia", "Lunguda"])]

data["user_ids_num"] = pd.factorize(data["user_ids"])[0]

In [31]:
# select 2 utterances per user_id as test set
from collections import defaultdict

# shuffle
data_sh = data.sample(frac=1, random_state=0).reset_index(drop=True)

user_id_map = defaultdict(int)
test_data_seen = []
train_val_data_seen = []

for i, item in data_sh.iterrows():
    
    # Only test on high speakers
    if speakers_dict[item.user_ids] <= 300 or user_id_map[item.user_ids] == 2:
        train_val_data_seen.append(item)
        continue
    
    # if item.duration >= 1.0: # speaker embedding need 1 sec for extraction.
    test_data_seen.append(item)
    user_id_map[item.user_ids]+=1
    
test_data_seen = pd.DataFrame(test_data_seen)
train_val_data_seen = pd.DataFrame(train_val_data_seen)

In [32]:
len(test_data_seen), len(train_val_data_seen), len(test_data_seen)+len(train_val_data_seen), len(data_sh)


(646, 35333, 35979, 35979)

In [33]:
train_val_data_seen = train_val_data_seen.sample(frac=1, random_state=0).reset_index(drop=True)
val = train_val_data_seen[0: 200]
train = train_val_data_seen[200: ]

In [39]:
test_data_unseen.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/afritts-test-unseen.csv", index=False)

train.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/afritts-train.csv", index=False)
val.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/afritts-dev.csv", index=False)

test_data_seen.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/afritts-test-seen.csv", index=False)


Normalize sentences


In [40]:
test_data_unseen["transcript"] = test_data_unseen["transcript"].apply(tts_cleaner)

# train["transcript"] = train["transcript"].apply(tts_cleaner)
# val["transcript"] = val["transcript"].apply(tts_cleaner)
# test_data_seen["transcript"] = test_data_seen["transcript"].apply(tts_cleaner)

[NeMo W 2024-02-20 16:35:14 nemo_logging:349] /tmp/ipykernel_20292/362128996.py:1: SettingWithCopyWarning: 
    A value is trying to be set on a copy of a slice from a DataFrame.
    Try using .loc[row_indexer,col_indexer] = value instead
    
    See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
      test_data_unseen["transcript"] = test_data_unseen["transcript"].apply(tts_cleaner)
    


In [41]:
test_data_unseen.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/afritts-test-unseen-clean.csv", index=False)

# train.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/afritts-train-clean.csv", index=False)
# val.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/afritts-dev-clean.csv", index=False)

# test_data_seen.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/afritts-test-unseen-clean.csv", index=False)

In [ ]:
# data.to_csv("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/data/temp/intron-tts-all-public-0000", index=False)

In [ ]:
with open("/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/vits/filelists/afritts_audio_train.txt", "w+") as f4:
    for i, item in train.iterrows():
        print(f"DUMMY3/{item.audio_ids}.wav|{item.user_ids_num}|{item.sentence}", file=f4)

In [ ]:
# select 1 sentence per accent per region for objective evaluation



In [ ]:
# Test set: 10 speakers seen in training

In [ ]:
speakers.tail(20)

In [ ]:
import librosa
import IPython.display as ipd

In [ ]:
data[0:1].audio_paths.values.item()

In [ ]:
data[data.audio_paths == '/AfriSpeech-TTS/train/9b1a7865-4b74-486e-8223-9117e2ea592a/9b1f7d24a96824967ed3a1ae5d3c44cd_OWVB6wln.wav']

In [ ]:
'/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/../AfriSpeech-TTS/train/9b1a7865-4b74-486e-8223-9117e2ea592a/9b1f7d24a96824967ed3a1ae5d3c44cd_OWVB6wln.wav'


In [ ]:
filepath = '/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/afrispeech_16k/1e759a52b9a7588771a8d281dd480fce.wav'

audio, sr = librosa.load(filepath, sr=16000)

ipd.Audio(audio, rate=sr)

In [ ]:
filepath = '/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/train/9b1a7865-4b74-486e-8223-9117e2ea592a/9b1f7d24a96824967ed3a1ae5d3c44cd_OWVB6wln.wav'

audio, sr = librosa.load(filepath, sr=16000)

ipd.Audio(audio, rate=sr)

In [ ]:
audio.shape

In [ ]:
sr

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import librosa.display

# parameters
n_fft=1024
hop_length=256
window="hann"
# sr=16000
n_mels=80
fmin=0
fmax=8000
win_length=n_fft
power=1.0
n_iter=64

def plot_spec(spec, aspect=2):
    if spec is not None:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        im = plt.imshow(spec, origin="lower",)
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label("Colorbar")
        ax.set_aspect(aspect)
        plt.show()

In [ ]:
audio.shape

In [ ]:
filepath = '/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/b53792d173f1ac743819e81f240a3a07_ooKIycx6.wav'

audio, sr = librosa.load(filepath, sr=16000)

ref_mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, S=None, n_fft=n_fft, 
                                          hop_length=hop_length, 
                                          win_length=win_length, 
                                          window=window, power=power,
                                          n_mels=n_mels, fmin=fmin, fmax=fmax,)
# vocoder expects a log mel-spectrogram
eps = 1e-9
ref_log_mel = np.log(ref_mel_spec + eps)

In [ ]:
ipd.Audio(audio, rate=sr)

In [ ]:
plot_spec(ref_log_mel,)

In [ ]:
# denoised
filepath = '/srv/storage/talc2@talc-data2.nancy/multispeech/calcul/users/sogun/AfriSpeech-TTS/b53792d173f1ac743819e81f240a3a07_ooKIycx6_dn.wav'

audio_dn, sr = librosa.load(filepath)

dn_mel_spec = librosa.feature.melspectrogram(y=audio_dn, sr=sr, S=None, n_fft=n_fft, 
                                          hop_length=hop_length, 
                                          win_length=win_length, 
                                          window=window, power=power,
                                          n_mels=n_mels, fmin=fmin, fmax=fmax,)
# vocoder expects a log mel-spectrogram
eps = 1e-9
dn_log_mel = np.log(dn_mel_spec + eps)

In [ ]:
ipd.Audio(audio, rate=sr)

In [ ]:
plot_spec(dn_log_mel,2.5)